<a href="https://colab.research.google.com/github/skilove13/Tensorflow-in-Practice-Specialization/blob/master/iris_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from keras.callbacks import ModelCheckpoint

test_data, train_data = tfds.load(
    name='iris',
    split=('train[:20%]','train[20%:]'),
    as_supervised=False)

train_data_len = len(list(train_data.as_numpy_iterator()))
test_data_len = len(list(test_data.as_numpy_iterator()))

def preprocess(dataset):
  features = tf.unstack(dataset['features'])
  labels = dataset['label']

  f = features
  I = tf.one_hot(labels,3)
  return f, I 

def solution_model():
  train_examples_batch, train_labels_batch = next(iter(train_data.map(preprocess).batch(train_data_len)))
  test_examples_batch , test_labels_batch = next(iter(test_data.map(preprocess).batch(test_data_len)))

  #모델 설계 
  model = keras.Sequential()
  model.add(keras.layers.Dense(32,activation='relu'))
  model.add(keras.layers.Dense(16, activation='relu'))
  model.add(keras.layers.Dense(3, activation='softmax'))

  #모델 컴파일 
  model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

  # 훈련 체크포인트 설계
  checkpoint_path = 'iris_chkpnt.ckpt'
  checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                  save_weights_only=True,
                                                  save_best_only=True,
                                                  monitor='val_loss',
                                                  verbose=1)
  
  #모델 학습
  model.fit(train_examples_batch, train_labels_batch,
            validation_data = (test_examples_batch, test_labels_batch),
            steps_per_epoch = train_data_len,
            validation_steps = test_data_len,
            callbacks=[checkpoint],
            epochs=50)

  model.load_weights(checkpoint_path)

  #모델 평가 
  results = model.evaluate(test_examples_batch, test_labels_batch, verbose=2)
  print("test loss , test acc:", results)

  return model 

# Note that you'll need to save your model as a .h5 like this
# This .h5 will be uploaded to the testing infrastructure
# and a score will be returned to you
if __name__ == '__main__':
    model = solution_model()
    model.save("irisModel.h5")
    

Epoch 1/50
107/120 [=========================>....] - ETA: 0s - loss: 0.8134 - accuracy: 0.6822
Epoch 00001: val_loss improved from inf to 0.58947, saving model to iris_chkpnt.ckpt
120/120 [==============================] - 0s 4ms/step - loss: 0.7965 - accuracy: 0.7000 - val_loss: 0.5895 - val_accuracy: 0.8667
Epoch 2/50
104/120 [=========================>....] - ETA: 0s - loss: 0.5135 - accuracy: 0.8077
Epoch 00002: val_loss improved from 0.58947 to 0.40763, saving model to iris_chkpnt.ckpt
120/120 [==============================] - 0s 3ms/step - loss: 0.5133 - accuracy: 0.8167 - val_loss: 0.4076 - val_accuracy: 0.9000
Epoch 3/50
112/120 [===========================>..] - ETA: 0s - loss: 0.4150 - accuracy: 0.8750
Epoch 00003: val_loss improved from 0.40763 to 0.31744, saving model to iris_chkpnt.ckpt
120/120 [==============================] - 0s 3ms/step - loss: 0.4060 - accuracy: 0.8833 - val_loss: 0.3174 - val_accuracy: 0.8667
Epoch 4/50
118/120 [============================>.] - ET